# 🍅 Tomato Segmentation Using U-Net

##### This code falls under the Assignment 2 of the course GIT 1 @ College of Computing - UM6P. 
##### Done by: 
- BADDOU Mounia (@MTheCreator), 
- FRI Zyad (@ZyadFri), 
- KABLY Malak (@malakkbl), 

##### Academic year: 2024 / 2025.

## Importing first all needed libraries

In [13]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import random
from sklearn.model_selection import train_test_split

# For reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

## Declaration of used datasets

In [14]:
# Define paths to your dataset folders
dataset_path = "Tomato_dataset"
train_path = os.path.join(dataset_path, "Train")
train2_path = os.path.join(dataset_path, "Train2")
mask_path = os.path.join(dataset_path, "Mask")
mask2_path = os.path.join(dataset_path, "Mask2")
test_path = os.path.join(dataset_path, "Test")
test2_path = os.path.join(dataset_path, "Test2")

# Image size - smaller size for faster training on CPU
IMG_SIZE = 256

## Definition of the handler class of our tomato data

In [15]:
class TomatoDataset(Dataset):
    """Custom Dataset for loading tomato images and their segmentation masks"""
    
    def __init__(self, img_dir, mask_dir, transform=None):
        """
        Args:
            img_dir (string): Directory with all the images
            mask_dir (string): Directory with all the masks
            transform (callable, optional): Optional transform to be applied on a sample
        """
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        
        # Get list of files in both directories
        self.img_names = sorted([f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
        self.mask_names = sorted([f for f in os.listdir(mask_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
        
        # Handle the case where the number of images and masks don't match
        if len(self.img_names) != len(self.mask_names):
            print(f"Warning: Number of images ({len(self.img_names)}) doesn't match number of masks ({len(self.mask_names)})")
            print("Will use the minimum number of files and assume they correspond to each other.")
            
            # Use only the common part based on filenames or indices
            if len(self.img_names) > len(self.mask_names):
                self.img_names = self.img_names[:len(self.mask_names)]
            else:
                self.mask_names = self.mask_names[:len(self.img_names)]
            
            print(f"Adjusted dataset size: {len(self.img_names)} images and masks")
    
    def __len__(self):
        return len(self.img_names)
    
    def __getitem__(self, idx):
        # Load image
        img_name = os.path.join(self.img_dir, self.img_names[idx])
        image = Image.open(img_name).convert("RGB")
        
        # Load mask
        mask_name = os.path.join(self.mask_dir, self.mask_names[idx])
        mask = Image.open(mask_name).convert("L")  # Convert to grayscale
        
        # Apply transformations if any
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
        
        # Normalize the mask to binary (0 for background, 1 for tomato)
        mask = (mask > 0.5).float()
        
        return image, mask

## Definition of the model, its architecture and its meta-parameters

In [16]:
# Define the U-Net model architecture
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        """
        U-Net architecture with fewer features for CPU efficiency
        Args:
            in_channels (int): Number of input channels (3 for RGB)
            out_channels (int): Number of output channels (1 for binary segmentation)
        """
        super(UNet, self).__init__()
        
        # Encoder (downsampling)
        # Using smaller feature maps than typical U-Net for CPU efficiency
        self.enc1 = self._double_conv(in_channels, 32)  # Original U-Net uses 64
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.enc2 = self._double_conv(32, 64)  # Original uses 128
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.enc3 = self._double_conv(64, 128)  # Original uses 256
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.enc4 = self._double_conv(128, 256)  # Original uses 512
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Bottleneck
        self.bottleneck = self._double_conv(256, 512)  # Original uses 1024
        
        # Decoder (upsampling)
        self.up_conv4 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec4 = self._double_conv(512, 256)  # 512 = 256 (from up_conv4) + 256 (skip connection from enc4)
        
        self.up_conv3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec3 = self._double_conv(256, 128)  # 256 = 128 (from up_conv3) + 128 (skip connection from enc3)
        
        self.up_conv2 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec2 = self._double_conv(128, 64)  # 128 = 64 (from up_conv2) + 64 (skip connection from enc2)
        
        self.up_conv1 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.dec1 = self._double_conv(64, 32)  # 64 = 32 (from up_conv1) + 32 (skip connection from enc1)
        
        # Output layer
        self.out_conv = nn.Conv2d(32, out_channels, kernel_size=1)
        self.sigmoid = nn.Sigmoid()
    
    def _double_conv(self, in_channels, out_channels):
        """Helper function for creating double convolution blocks"""
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        # Encoder path
        enc1_out = self.enc1(x)
        enc2_out = self.enc2(self.pool1(enc1_out))
        enc3_out = self.enc3(self.pool2(enc2_out))
        enc4_out = self.enc4(self.pool3(enc3_out))
        
        # Bottleneck
        bottleneck_out = self.bottleneck(self.pool4(enc4_out))
        
        # Decoder path with skip connections
        dec4_out = self.up_conv4(bottleneck_out)
        # Ensure dimensions match before concatenation (may be necessary if odd dimensions)
        if dec4_out.shape != enc4_out.shape[0:2]:
            enc4_out = enc4_out[:, :, :dec4_out.shape[2], :dec4_out.shape[3]]
        dec4_out = self.dec4(torch.cat([dec4_out, enc4_out], dim=1))
        
        dec3_out = self.up_conv3(dec4_out)
        # Ensure dimensions match
        if dec3_out.shape[2:] != enc3_out.shape[2:]:
            enc3_out = enc3_out[:, :, :dec3_out.shape[2], :dec3_out.shape[3]]
        dec3_out = self.dec3(torch.cat([dec3_out, enc3_out], dim=1))
        
        dec2_out = self.up_conv2(dec3_out)
        # Ensure dimensions match
        if dec2_out.shape[2:] != enc2_out.shape[2:]:
            enc2_out = enc2_out[:, :, :dec2_out.shape[2], :dec2_out.shape[3]]
        dec2_out = self.dec2(torch.cat([dec2_out, enc2_out], dim=1))
        
        dec1_out = self.up_conv1(dec2_out)
        # Ensure dimensions match
        if dec1_out.shape[2:] != enc1_out.shape[2:]:
            enc1_out = enc1_out[:, :, :dec1_out.shape[2], :dec1_out.shape[3]]
        dec1_out = self.dec1(torch.cat([dec1_out, enc1_out], dim=1))
        
        # Output layer
        out = self.out_conv(dec1_out)
        return self.sigmoid(out)

## Definition of helper functions in treating the images according to what was seen in the course

In [17]:
# Data augmentation and preprocessing
def get_transforms():
    """Define transformations for images and masks"""
    transform = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        # Only normalize the images, not the masks
    ])
    
    return transform

# Dice coefficient for evaluation
def dice_coefficient(pred, target):
    """
    Calculate the Dice coefficient between prediction and target
    
    Args:
        pred (torch.Tensor): Predicted binary mask
        target (torch.Tensor): Ground truth binary mask
    
    Returns:
        float: Dice coefficient value
    """
    smooth = 1e-5  # To avoid division by zero
    
    # Flatten the tensors
    pred_flat = pred.view(-1)
    target_flat = target.view(-1)
    
    # Calculate intersection and union
    intersection = (pred_flat * target_flat).sum()
    union = pred_flat.sum() + target_flat.sum()
    
    # Calculate Dice coefficient
    dice = (2. * intersection + smooth) / (union + smooth)
    
    return dice.item()

# IoU (Intersection over Union) / Jaccard Index
def iou_score(pred, target):
    """
    Calculate IoU (Intersection over Union) between prediction and target
    
    Args:
        pred (torch.Tensor): Predicted binary mask
        target (torch.Tensor): Ground truth binary mask
    
    Returns:
        float: IoU score
    """
    smooth = 1e-5
    
    # Flatten the tensors
    pred_flat = pred.view(-1)
    target_flat = target.view(-1)
    
    # Calculate intersection and union
    intersection = (pred_flat * target_flat).sum()
    union = pred_flat.sum() + target_flat.sum() - intersection
    
    # Calculate IoU
    iou = (intersection + smooth) / (union + smooth)
    
    return iou.item()


def visualize_prediction(image, true_mask, pred_mask, idx=0, save_path=None):
    """
    Visualize original image, ground truth mask, and predicted mask
    
    Args:
        image (torch.Tensor): Input image
        true_mask (torch.Tensor): Ground truth mask
        pred_mask (torch.Tensor): Predicted mask
        idx (int): Sample index
        save_path (str): Path to save the visualization
    """
    # Convert tensors to numpy arrays
    img = image.permute(1, 2, 0).cpu().numpy()
    true_mask = true_mask.squeeze().cpu().numpy()
    pred_mask = pred_mask.squeeze().cpu().numpy()
    
    # Create a figure with 3 subplots
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Plot original image
    axes[0].imshow(img)
    axes[0].set_title("Original Image")
    axes[0].axis("off")
    
    # Plot ground truth mask
    axes[1].imshow(true_mask, cmap="gray")
    axes[1].set_title("Ground Truth Mask")
    axes[1].axis("off")
    
    # Plot predicted mask
    axes[2].imshow(pred_mask, cmap="gray")
    axes[2].set_title(f"Predicted Mask\nDice: {dice_coefficient(torch.tensor(pred_mask), torch.tensor(true_mask)):.4f}")
    axes[2].axis("off")
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path)
    else:
        plt.show()
    
    plt.close()

## Definition of Training and Evaluating functions of our model

In [18]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device="cpu"):
    """
    Train the U-Net model
    
    Args:
        model (nn.Module): U-Net model
        train_loader (DataLoader): Training data loader
        val_loader (DataLoader): Validation data loader
        criterion: Loss function
        optimizer: Optimizer
        num_epochs (int): Number of training epochs
        device (str): Device to train on ('cpu' or 'cuda')
    
    Returns:
        model: Trained model
        dict: Training history (loss, accuracy, etc.)
    """
    # Initialize variables to store metrics
    history = {
        'train_loss': [],
        'val_loss': [],
        'train_dice': [],
        'val_dice': [],
        'train_iou': [],
        'val_iou': []
    }
    
    # Start timer
    start_time = time.time()
    
    # Training loop
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        
        # Training phase
        model.train()
        train_loss = 0.0
        train_dice = 0.0
        train_iou = 0.0
        
        # Use tqdm for progress bar
        for images, masks in tqdm(train_loader, desc="Training"):
            # Move data to device
            images = images.to(device)
            masks = masks.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Update metrics
            train_loss += loss.item()
            
            # Convert predictions to binary (0 or 1)
            preds = (outputs > 0.5).float()
            
            # Calculate Dice coefficient and IoU
            train_dice += dice_coefficient(preds, masks)
            train_iou += iou_score(preds, masks)
        
        # Calculate average metrics for this epoch
        train_loss /= len(train_loader)
        train_dice /= len(train_loader)
        train_iou /= len(train_loader)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_dice = 0.0
        val_iou = 0.0
        
        with torch.no_grad():
            for images, masks in tqdm(val_loader, desc="Validation"):
                images = images.to(device)
                masks = masks.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, masks)
                
                val_loss += loss.item()
                
                preds = (outputs > 0.5).float()
                val_dice += dice_coefficient(preds, masks)
                val_iou += iou_score(preds, masks)
        
        val_loss /= len(val_loader)
        val_dice /= len(val_loader)
        val_iou /= len(val_loader)
        
        # Store metrics in history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_dice'].append(train_dice)
        history['val_dice'].append(val_dice)
        history['train_iou'].append(train_iou)
        history['val_iou'].append(val_iou)
        
        # Print metrics
        print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        print(f"Train Dice: {train_dice:.4f}, Val Dice: {val_dice:.4f}")
        print(f"Train IoU: {train_iou:.4f}, Val IoU: {val_iou:.4f}")
        
        # Print estimated time remaining
        elapsed_time = time.time() - start_time
        estimated_total_time = elapsed_time * num_epochs / (epoch + 1)
        estimated_remaining_time = estimated_total_time - elapsed_time
        print(f"Elapsed time: {elapsed_time/60:.2f} min, Est. remaining: {estimated_remaining_time/60:.2f} min")
        print("-" * 50)
    
    # Total training time
    total_time = time.time() - start_time
    print(f"Total training time: {total_time/60:.2f} minutes")
    
    return model, history

def evaluate_model(model, test_loader, criterion, device="cpu"):
    """
    Evaluate the trained model on test data
    
    Args:
        model (nn.Module): Trained U-Net model
        test_loader (DataLoader): Test data loader
        criterion: Loss function
        device (str): Device to evaluate on ('cpu' or 'cuda')
    
    Returns:
        dict: Evaluation metrics
    """
    model.eval()
    test_loss = 0.0
    test_dice = 0.0
    test_iou = 0.0
    
    # Lists to store images and masks for visualization
    test_images = []
    test_true_masks = []
    test_pred_masks = []
    
    with torch.no_grad():
        for images, masks in tqdm(test_loader, desc="Testing"):
            images = images.to(device)
            masks = masks.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            test_loss += loss.item()
            
            preds = (outputs > 0.5).float()
            test_dice += dice_coefficient(preds, masks)
            test_iou += iou_score(preds, masks)
            
            # Store some samples for visualization (first 5)
            if len(test_images) < 5:
                test_images.append(images[0])
                test_true_masks.append(masks[0])
                test_pred_masks.append(preds[0])
    
    test_loss /= len(test_loader)
    test_dice /= len(test_loader)
    test_iou /= len(test_loader)
    
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Dice: {test_dice:.4f}")
    print(f"Test IoU: {test_iou:.4f}")
    
    # Create output directory for visualizations
    os.makedirs("results", exist_ok=True)
    
    # Visualize predictions
    for i in range(len(test_images)):
        visualize_prediction(
            test_images[i], 
            test_true_masks[i], 
            test_pred_masks[i], 
            idx=i, 
            save_path=f"results/prediction_{i}.png"
        )
    
    return {
        'loss': test_loss,
        'dice': test_dice,
        'iou': test_iou,
        'images': test_images,
        'true_masks': test_true_masks,
        'pred_masks': test_pred_masks
    }

## Function responsible to plot our output 

In [19]:
def plot_training_history(history):
    """
    Plot training history
    
    Args:
        history (dict): Training history dictionary
    """
    # Create output directory
    os.makedirs("results", exist_ok=True)
    
    # Plot loss
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot Dice coefficient
    plt.subplot(1, 2, 2)
    plt.plot(history['train_dice'], label='Train Dice')
    plt.plot(history['val_dice'], label='Validation Dice')
    plt.title('Dice Coefficient')
    plt.xlabel('Epoch')
    plt.ylabel('Dice')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig("results/training_history.png")
    plt.close()
    
    # Plot IoU
    plt.figure(figsize=(8, 4))
    plt.plot(history['train_iou'], label='Train IoU')
    plt.plot(history['val_iou'], label='Validation IoU')
    plt.title('IoU Score')
    plt.xlabel('Epoch')
    plt.ylabel('IoU')
    plt.legend()
    plt.tight_layout()
    plt.savefig("results/iou_history.png")
    plt.close()

def segment_new_image(model, image_path, device="cpu"):
    """
    Segment a new unseen image
    
    Args:
        model (nn.Module): Trained U-Net model
        image_path (str): Path to the image
        device (str): Device to run on ('cpu' or 'cuda')
    
    Returns:
        numpy.ndarray: Original image
        numpy.ndarray: Segmented mask
    """
    # Load and preprocess the image
    transform = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
    ])
    
    image = Image.open(image_path).convert("RGB")
    original_size = image.size
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # Get prediction
    model.eval()
    with torch.no_grad():
        output = model(image_tensor)
        pred_mask = (output > 0.5).float()
    
    # Convert to numpy arrays
    image_np = image_tensor.squeeze().permute(1, 2, 0).cpu().numpy()
    mask_np = pred_mask.squeeze().cpu().numpy()
    
    # Create output directory
    os.makedirs("results", exist_ok=True)
    
    # Visualize
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(image_np)
    plt.title("Original Image")
    plt.axis("off")
    
    plt.subplot(1, 2, 2)
    plt.imshow(mask_np, cmap="gray")
    plt.title("Segmented Mask")
    plt.axis("off")
    
    plt.tight_layout()
    plt.savefig("results/new_image_segmentation.png")
    plt.close()
    
    return image_np, mask_np

## Main function to orchestrate the use of everything defined beforehands

In [20]:
def main():
    # Check if GPU is available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Create transformations
    transform = get_transforms()
    
    # Create datasets
    print("Loading datasets...")
    
    # Load training images from both Train and Train2 folders with their corresponding masks
    train_dataset1 = TomatoDataset(
        img_dir=train_path,
        mask_dir=mask_path,
        transform=transform
    )
    
    train_dataset2 = TomatoDataset(
        img_dir=train2_path,
        mask_dir=mask2_path,
        transform=transform
    )
    
    # Combine both training datasets
    full_train_dataset = torch.utils.data.ConcatDataset([train_dataset1, train_dataset2])
    
    # Split into train and validation (80/20 split)
    train_size = int(0.8 * len(full_train_dataset))
    val_size = len(full_train_dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        full_train_dataset, [train_size, val_size]
    )
    
    # Load test datasets from both Test and Test2 folders
    test_dataset1 = TomatoDataset(
        img_dir=test_path,
        mask_dir=mask_path,  # Using Mask for Test folder
        transform=transform
    )
    
    test_dataset2 = TomatoDataset(
        img_dir=test2_path,
        mask_dir=mask2_path,  # Using Mask2 for Test2 folder
        transform=transform
    )
    
    # Combine both test datasets
    test_dataset = torch.utils.data.ConcatDataset([test_dataset1, test_dataset2])
    
    print(f"Train dataset size: {len(train_dataset)}")
    print(f"Validation dataset size: {len(val_dataset)}")
    print(f"Test dataset size: {len(test_dataset)}")
    
    # Use smaller batch size for CPU training
    batch_size = 4
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Display a sample from the training set
    sample_img, sample_mask = next(iter(train_loader))
    print(f"Sample image shape: {sample_img.shape}")
    print(f"Sample mask shape: {sample_mask.shape}")
    
    # Visualize a sample
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(sample_img[0].permute(1, 2, 0).numpy())
    plt.title("Sample Image")
    plt.axis("off")
    
    plt.subplot(1, 2, 2)
    plt.imshow(sample_mask[0].squeeze().numpy(), cmap="gray")
    plt.title("Sample Mask")
    plt.axis("off")
    
    plt.tight_layout()
    os.makedirs("results", exist_ok=True)
    plt.savefig("results/sample_data.png")
    plt.close()
    
    # Create U-Net model
    model = UNet(in_channels=3, out_channels=1).to(device)
    print(model)
    
    # Define loss function and optimizer
    # Using BCELoss for binary segmentation
    criterion = nn.BCELoss()
    
    # Using Adam optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Train the model (reduced epochs for CPU training)
    num_epochs = 20
    print(f"Training for {num_epochs} epochs on {device}...")
    print("Note: This will take several hours on CPU. Consider reducing the number of epochs or image size.")
    
    trained_model, history = train_model(
        model, train_loader, val_loader, criterion, optimizer, 
        num_epochs=num_epochs, device=device
    )
    
    # Plot training history
    plot_training_history(history)
    
    # Evaluate on test dataset
    print("Evaluating on test dataset...")
    test_results = evaluate_model(trained_model, test_loader, criterion, device=device)
    
    # Save the model
    torch.save(trained_model.state_dict(), "results/tomato_unet_model.pth")
    print("Model saved successfully.")
    
    # If any test images are available in the Test folder, segment them
    test_files = [f for f in os.listdir(test_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
    if test_files:
        print("Segmenting a test image...")
        test_img_path = os.path.join(test_path, test_files[0])
        segment_new_image(trained_model, test_img_path, device=device)


main()

Using device: cpu
Loading datasets...
Will use the minimum number of files and assume they correspond to each other.
Adjusted dataset size: 20 images and masks
Will use the minimum number of files and assume they correspond to each other.
Adjusted dataset size: 20 images and masks
Train dataset size: 80
Validation dataset size: 20
Test dataset size: 40
Sample image shape: torch.Size([4, 3, 256, 256])
Sample mask shape: torch.Size([4, 1, 256, 256])
UNet(
  (enc1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (enc2): Sequential(
    (0): Conv2d(32, 64, k

Validation: 100%|██████████| 5/5 [00:02<00:00,  1.89it/s]


Train Loss: 0.4004, Val Loss: 0.4188
Train Dice: 0.7820, Val Dice: 0.5602
Train IoU: 0.6691, Val IoU: 0.3990
Elapsed time: 0.57 min, Est. remaining: 10.91 min
--------------------------------------------------
Epoch 2/20


Validation: 100%|██████████| 5/5 [00:02<00:00,  1.88it/s]


Train Loss: 0.3036, Val Loss: 0.2633
Train Dice: 0.8207, Val Dice: 0.8350
Train IoU: 0.7129, Val IoU: 0.7230
Elapsed time: 1.15 min, Est. remaining: 10.32 min
--------------------------------------------------
Epoch 3/20


...

Train Loss: 0.1528, Val Loss: 0.1495
Train Dice: 0.8277, Val Dice: 0.8843
Train IoU: 0.7241, Val IoU: 0.7966
Elapsed time: 10.75 min, Est. remaining: 0.57 min
--------------------------------------------------
Epoch 20/20


Validation: 100%|██████████| 5/5 [00:02<00:00,  1.93it/s]


Train Loss: 0.1451, Val Loss: 0.1274
Train Dice: 0.8366, Val Dice: 0.8837
Train IoU: 0.7367, Val IoU: 0.7959
Elapsed time: 11.31 min, Est. remaining: 0.00 min
--------------------------------------------------
Total training time: 11.31 minutes
Evaluating on test dataset...


Testing: 100%|██████████| 10/10 [00:05<00:00,  1.87it/s]


Test Loss: 0.2280
Test Dice: 0.7720
Test IoU: 0.6576
Model saved successfully.
Segmenting a test image...
